This is the beginning of the Toronto clustering and analysis project.  Below, the data is scraped from the Wikepedia page.

In [1]:
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import lxml
import html5lib
from bs4 import BeautifulSoup
import requests

The following cells are the first task of Unit 3 of Course 9 of the series. This scrapes the html file off the website, then loops on the table, identified in the html by the collection of 'tr's.  Each tr for data has 3 td's in it, the Postal Code, the Borough and the Neighborhood, in that order.  These are assigned to a list during each cycle and the list populates a dictionary.  The dictionary is used to create a dataframe, which, in turn, is appended to the Toronto dataframe, tordf.  The responses to 'Not assigned' are also included in the loop. That completes the process of creating the basic dataframe.

In [2]:
html_file=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(html_file, 'lxml')
table=soup.find('table')
#print (table)
tordf=pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'],dtype=str)
ent=['a','b','c']
pdlib={'Postalcode':'a','Borough':'b','Neighborhood':'c'}
j=1
for tr in table.find_all('tr'):
    i=0
    for td in tr.find_all('td'):
        ent[i]=td.text
        i=i+1
    if ent[1]=='Not assigned':
        continue;
    if ent[0]=='a':
        continue;
    ent[2]=ent[2].replace('\n',"")
    if ent[2]=='Not assigned':
        ent[2]=ent[1]
    pdlib['Neighborhood']=ent[2]
    pdlib['Postalcode']=ent[0]
    pdlib['Borough']=ent[1]
    tordf=tordf.append([pdlib],ignore_index=True)
tordf.head(10)


,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Now, to eliminate Postal Code duplication by combining neighborhoods in one Postal Code row. A dataframe is created using Groupby, grouping on Postal Code and Borough together. Commas are inserted into the Neighborhood entries via an .apply to the join of the data. A reset_index is used to make the index of the original groupby become the first two columns, i.e. Postalcode and Borough. The original Groupby sorted the Postal Codes.

In [3]:
tordf2=pd.DataFrame(tordf.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join),columns=['Neighborhood']).reset_index()
tordf2.head(10)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [4]:
tordf2.shape

(103, 3)

103 Postal Codes, each with the Borough and the list of Neighborhoods.

This completes the first part of the assignment for Unit 3 of Course 9.

The following is the second part of the assignment for Unit 3 of Course 9. The next step in the project is to add longitude and latitude columns to the tordf2 df. Looping through the Postal Codes will be necessary, creating a pair of lists.  These lists are then assigned to new columns in tordf2.

In [5]:
!pip install geocoder
import geocoder # import geocoder


    100% |████████████████████████████████| 102kB 7.3MB/s ta 0:00:01
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: decorator in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from ratelim->geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Pyt

I never got the geocoder to work.  The loop sought eternally.  So, on to the csv file.  That will require loading it, sorting it, then finding the latitude and longitude for the Postal Code.

In [6]:
dset=pd.read_csv("http://cocl.us/Geospatial_data")
print(dset.head(10))
dset.shape

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
5         M1J  43.744734 -79.239476
6         M1K  43.727929 -79.262029
7         M1L  43.711112 -79.284577
8         M1M  43.716316 -79.239476
9         M1N  43.692657 -79.264848


(103, 3)

The latitude and longitude list is in alphabetical order by Postal Code, just like tordf2.  The length is also the same; all the Postal Codes are included. One only needs to copy the latitude and longitude columns onto the tordf2 dataframe.

In [7]:
tordf2['Latitude']=dset['Latitude']
tordf2['Longitude']=dset['Longitude']
tordf2.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
print('The dataframe has {} boroughs and {} postal codes.'.format(
        len(tordf2['Borough'].unique()),
        tordf2.shape[0]
    )
)

The dataframe has 11 boroughs and 103 postal codes.


This concludes the 2nd section of the assignment for Unit 3 of Course 9.

In [ ]:
The following is the beginning of the 3rd section of 

Bring in the libraries needed to cluster and map the borough and neighnorhood data.

In [9]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare 
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.21 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.70 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  40.05 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  45.82 MB/s
Libraries imported.


Make an overview map that shows all the postal codes in the boroughs.  Use Nominatim, folium and geolocator.

In [10]:

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
    # create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Postalcode in zip(tordf2['Latitude'], tordf2['Longitude'], tordf2['Borough'], tordf2['Postalcode']):
    label = '{}, {}'.format(Postalcode, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto

The geographical coordinate of Toronto are 43.653963, -79.387207.


Choose the borough of Scarborough for examination.  First, create a dataframe by extracting the Scarborough data from tordf2, then make a map of the borough.

In [11]:
scardf=tordf2[tordf2['Borough']=='Scarborough'].reset_index(drop=True)
print(scardf.head())
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="sc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough Ontario are {}, {}.'.format(latitude, longitude))
print('Scarborough has', scardf.shape[0],'Postal Codes.')

  Postalcode      Borough                            Neighborhood   Latitude  \
0        M1B  Scarborough                          Rouge, Malvern  43.806686   
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2        M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3        M1G  Scarborough                                  Woburn  43.770992   
4        M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  
The geograpical coordinate of Scarborough Ontario are 43.773077, -79.257774.
Scarborough has 17 Postal Codes.


In [12]:
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, Postalcode, neighborhood in zip(scardf['Latitude'], scardf['Longitude'], scardf['Postalcode'], scardf['Neighborhood']):
    label = '{}, {}'.format(Postalcode,neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
map_scarborough

Explore Scarborough for venues.  Start by establishingh access info for using Foursquare.

In [13]:
CLIENT_ID = 'VCB1F2GAXJNP0ECNYPZ2MILKJ0EY0N4VQPKAHEBNYW4YSQI4' # your Foursquare ID
CLIENT_SECRET = 'MMDYZKGTQCPHKK0JYZGBHFSRDZ5ZNLB5TUNX4MVGITNBYNNR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


I'll randomly choose a postal code from the 17 in Scarborough and examine that area for venues. Just choose a value for dahood at the beginning of the code.

In [14]:
dahood=5
neighborhood_latitude = scardf.loc[dahood, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scardf.loc[dahood, 'Longitude'] # neighborhood longitude value

neighborhood_name = scardf.loc[dahood, 'Neighborhood'] # neighborhood name
postalcode = scardf.loc[dahood,'Postalcode']
print('Latitude and longitude values of {} are {}, {}. The Postal Code is {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude,
                                                               postalcode))
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
import requests
results = requests.get(url).json()
print('There are {} venues around the neighborhood of {}.'.format(len(results['response']['groups'][0]['items']),neighborhood_name))

Latitude and longitude values of Scarborough Village are 43.7447342, -79.23947609999999. The Postal Code is M1J.
There are 42 venues around the neighborhood of Scarborough Village.


Get the json and start working on it. Extract the venues for all the neighborhoods in Scarborough.

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6450154c1f6764eb2e528a'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5085ec39e4b0b1ead2eb0818-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/toys_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1f3941735',
         'name': 'Toy / Game Store',
         'pluralName': 'Toy / Game Stores',
         'primary': True,
         'shortName': 'Toys & Games'}],
       'id': '5085ec39e4b0b1ead2eb0818',
       'location': {'address': '300 Borough Drive',
        'cc': 'CA',
        'city': 'Scarborough',
        'country': 'Canada',
        'crossStreet': 'in Scarborough Town Centre',
        'distance': 284,
        'formattedAddress': ['300 Borough Drive (in Scarborough Town Centre)',
         'Scarborough ON M1P 4P5

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print(nearby_venues.head())
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

                           name        categories        lat        lng
0                  Disney Store  Toy / Game Store  43.775537 -79.256833
1  Tommy Hilfiger Company Store    Clothing Store  43.776015 -79.257369
2                     DAVIDsTEA          Tea Room  43.776613 -79.258516
3     American Eagle Outfitters    Clothing Store  43.775908 -79.258352
4                       SEPHORA    Cosmetics Shop  43.775592 -79.258242
42 venues were returned by Foursquare.


Routine to get nearby venues for a neighborhood.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


Get the venues for all the Scarborough neighborhoods. Create a dataframe with a line for each venue containing the latitude and longitude of the venue in the neighborhood.

In [18]:
rad=1500
LIMIT=100
Scarborough_venues = getNearbyVenues(names=scardf['Neighborhood'],
                                   latitudes=scardf['Latitude'],
                                   longitudes=scardf['Longitude'],
                                   radius=rad
                                    )
print(Scarborough_venues.shape)
Scarborough_venues.head()

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
(656, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,43.802283,-79.198565,Spa
1,"Rouge, Malvern",43.806686,-79.194353,Canadiana exhibit,43.817962,-79.193374,Zoo Exhibit
2,"Rouge, Malvern",43.806686,-79.194353,Caribbean Wave,43.798558,-79.195777,Caribbean Restaurant
3,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.802008,-79.198080,Fast Food Restaurant
4,"Rouge, Malvern",43.806686,-79.194353,LCBO,43.796671,-79.204586,Liquor Store


In [19]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,62,62,62,62,62,62
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",70,70,70,70,70,70
"Birch Cliff, Cliffside West",16,16,16,16,16,16
Cedarbrae,62,62,62,62,62,62
"Clairlea, Golden Mile, Oakridge",38,38,38,38,38,38
"Clarks Corners, Sullivan, Tam O'Shanter",47,47,47,47,47,47
"Cliffcrest, Cliffside, Scarborough Village West",41,41,41,41,41,41
"Dorset Park, Scarborough Town Centre, Wexford Heights",56,56,56,56,56,56
"East Birchmount Park, Ionview, Kennedy Park",38,38,38,38,38,38


Distribution of venues among neighborhoods.

In [20]:
# one hot encoding
scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Zoo Exhibit,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotel,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Music Store,National Park,Neighborhood,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Rock Club,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Normalize the distribution of venues.

In [21]:
print(scarborough_onehot.shape)
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
print(scarborough_grouped.shape)
scarborough_grouped


(656, 140)
(17, 140)


,Neighborhood,Zoo Exhibit,American Restaurant,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Big Box Store,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dumpling Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hotel,Hotpot Restaurant,Indian Chinese Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Lounge,Malay Restaurant,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Music Store,National Park,Noodle House,Optical Shop,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plaza,Pool,Pool Hall,Print Shop,Pub,Rental Car Location,Restaurant,Rock Club,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tea Room,Tennis Court,Thai Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.032258,0.016129,0.000000,0.00000,0.000000,0.000000,0.000000,0.016129,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.032258,0.032258,0.161290,0.000000,0.048387,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.032258,0.000000,0.016129,0.000000,0.000000,0.016129,0.016129,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.016129,0.000000,0.000000,0.000000,0.016129,0.000000,0.016129,0.016129,0.016129,0.000000,0.016129,0.016129,0.016129,0.000000,0.016129,0.016129,0.000000,0.016129,0.016129,0.016129,0.048387,0.016129,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.016129,0.000000,0.016129,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.028571,0.014286,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042857,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.014286,0.014286,0.185714,0.000000,0.057143,0.000000,0.000000,0.000000,0.000000,0.042857,0.014286,0.000000,0.028571,0.028571,0.000000,0.014286,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.014286,0.000000,0.000000,0.000000,0.000000,0.014286,0.0000

Identify the top 5 for each neighborhood.

In [22]:
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.16
1         Shopping Mall  0.05
2           Coffee Shop  0.05
3        Breakfast Spot  0.03
4  Hong Kong Restaurant  0.03


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0  Chinese Restaurant  0.19
1           BBQ Joint  0.07
2   Korean Restaurant  0.06
3         Coffee Shop  0.06
4         Pizza Place  0.04


----Birch Cliff, Cliffside West----
                   venue  freq
0                   Park  0.19
1            Golf Course  0.06
2                   Bank  0.06
3  General Entertainment  0.06
4                    Gym  0.06


----Cedarbrae----
               venue  freq
0        Coffee Shop  0.11
1  Indian Restaurant  0.05
2             Bakery  0.05
3     Clothing Store  0.05
4     Sandwich Place  0.05


----Clairlea, Golden Mile, Oakridge----
                  venue  freq
0           Coffee Shop  0.11
1         Grocery Store  0.08
2           Pizza Place  0.08

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

And the most common venues for each of the neighborhoods.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Shopping Mall,Coffee Shop,Cantonese Restaurant,Japanese Restaurant,Breakfast Spot,Hong Kong Restaurant,Caribbean Restaurant,Bakery,Gym / Fitness Center
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Chinese Restaurant,BBQ Joint,Coffee Shop,Korean Restaurant,Dessert Shop,Pharmacy,Pizza Place,Bubble Tea Shop,Tea Room,Noodle House
2,"Birch Cliff, Cliffside West",Park,Diner,Bank,Café,Filipino Restaurant,Fast Food Restaurant,General Entertainment,Restaurant,Golf Course,Thai Restaurant
3,Cedarbrae,Coffee Shop,Clothing Store,Sandwich Place,Bakery,Indian Restaurant,Pharmacy,Food Court,Fast Food Restaurant,Restaurant,Hotel
4,"Clairlea, Golden Mile, Oakridge",Coffee Shop,Grocery Store,Pizza Place,Burger Joint,Diner,Sandwich Place,Fast Food Restaurant,Pub,Food Truck,Bus Stop


In [39]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 4, 2, 2, 3, 2, 1, 2], dtype=int32)

In [40]:
# add clustering labels
print(kmeans.labels_)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scardf
print(scardf.shape, neighborhoods_venues_sorted.shape)
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#scarborough_merged[['Cluster Labels']]=scarborough_merged[['Cluster Labels']].astype('int32')
scarborough_merged.head(17) # check the last columns!


[0 0 0 2 2 0 0 2 2 2 4 2 2 3 2 1 2]
(17, 5) (17, 12)


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,3,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Coffee Shop,Spa,Women's Store,Greek Restaurant,Fruit & Vegetable Store,Liquor Store,Movie Theater
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,4,Pizza Place,Italian Restaurant,Burger Joint,Breakfast Spot,Gym,Grocery Store,Hotel,Event Space,Falafel Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,2,Pizza Place,Coffee Shop,Fast Food Restaurant,Gym / Fitness Center,Fried Chicken Joint,Discount Store,Park,Pharmacy,Sandwich Place,Liquor Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Pharmacy,Pizza Place,Indian Restaurant,Bakery,Fast Food Restaurant,Beer Store,Sandwich Place,Chinese Restaurant,Clothing Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2,Coffee Shop,Clothing Store,Sandwich Place,Bakery,Indian Restaurant,Pharmacy,Food Court,Fast Food Restaurant,Restaurant,Hotel
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Fast Food Restaurant,Sandwich Place,Pharmacy,Grocery Store,Wings Joint,Pizza Place,Coffee Shop,Breakfast Spot,Bus Station,Bookstore
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029,2,Coffee Shop,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Grocery Store,Train Station,Pharmacy,Pizza Place,Bus Stop,Bus Line
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577,2,Coffee Shop,Grocery Store,Pizza Place,Burger Joint,Diner,Sandwich Place,Fast Food Restaurant,Pub,Food Truck,Bus Stop
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476,0,Fast Food Restaurant,Harbor / Marina,Park,Pizza Place,Sandwich Place,Discount Store,Grocery Store,Beach,Pharmacy,Seafood Restaurant
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,0,Park,Diner,Bank,Café,Filipino Restaurant,Fast Food Restaurant,General Entertainment,Restaurant,Golf Course,Thai Restaurant


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Because some neighborhoods of Scarborough are remote, a larger radius (1,500 m) for Foursquare was necessary for finding venues for ALL the neighborhoods.

The radius problem is driven by Cluster 1, the village of Upper Rouge in Scarborough, in the extreme northeast, just northwest of a National Park.

Cluster 0 is a group of exurban neighborhoods combining commercial zones with developments like shopping malls and marinas with suburban neighborhoods.  This cluster is characterized by many fast food eating venues.

Cluster 2 is a suburban area with commercial development to service the residents.  Venues include pharmacies, restaraunts of various types and pet stores.

Cluster 3 is a short distance southwest of the Rouge Urban National Park and the color is difficult to see.  This cluster has traits similar to Cluster 1, but is closer to the suburban communities and their commercial venues.

Cluster 4 is near the lakeshore and has a combination residential and lakeshore commercial development with hotels, an event space and restaraunts.

In [33]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[0]+[2] + list(range(5, scarborough_merged.shape[1]))]]

,Postalcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M1M,"Cliffcrest, Cliffside, Scarborough Village West",0,Fast Food Restaurant,Harbor / Marina,Park,Pizza Place,Sandwich Place,Discount Store,Grocery Store,Beach,Pharmacy,Seafood Restaurant
9,M1N,"Birch Cliff, Cliffside West",0,Park,Diner,Bank,Café,Filipino Restaurant,Fast Food Restaurant,General Entertainment,Restaurant,Golf Course,Thai Restaurant
12,M1S,Agincourt,0,Chinese Restaurant,Shopping Mall,Coffee Shop,Cantonese Restaurant,Japanese Restaurant,Breakfast Spot,Hong Kong Restaurant,Caribbean Restaurant,Bakery,Gym / Fitness Center
13,M1T,"Clarks Corners, Sullivan, Tam O'Shanter",0,Fast Food Restaurant,Park,Chinese Restaurant,Coffee Shop,Cantonese Restaurant,Vietnamese Restaurant,Falafel Restaurant,Pharmacy,Pizza Place,Sandwich Place
14,M1V,"Agincourt North, L'Amoreaux East, Milliken, St...",0,Chinese Restaurant,BBQ Joint,Coffee Shop,Korean Restaurant,Dessert Shop,Pharmacy,Pizza Place,Bubble Tea Shop,Tea Room,Noodle House


In [34]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[0]+[2] + list(range(5, scarborough_merged.shape[1]))]]

,Postalcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,M1X,Upper Rouge,1,Farm,National Park,Fast Food Restaurant,Food & Drink Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Yoga Studio,Food Truck


In [35]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[0]+[2] + list(range(5, scarborough_merged.shape[1]))]]

,Postalcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M1E,"Guildwood, Morningside, West Hill",2,Pizza Place,Coffee Shop,Fast Food Restaurant,Gym / Fitness Center,Fried Chicken Joint,Discount Store,Park,Pharmacy,Sandwich Place,Liquor Store
3,M1G,Woburn,2,Coffee Shop,Pharmacy,Pizza Place,Indian Restaurant,Bakery,Fast Food Restaurant,Beer Store,Sandwich Place,Chinese Restaurant,Clothing Store
4,M1H,Cedarbrae,2,Coffee Shop,Clothing Store,Sandwich Place,Bakery,Indian Restaurant,Pharmacy,Food Court,Fast Food Restaurant,Restaurant,Hotel
5,M1J,Scarborough Village,2,Fast Food Restaurant,Sandwich Place,Pharmacy,Grocery Store,Wings Joint,Pizza Place,Coffee Shop,Breakfast Spot,Bus Station,Bookstore
6,M1K,"East Birchmount Park, Ionview, Kennedy Park",2,Coffee Shop,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Grocery Store,Train Station,Pharmacy,Pizza Place,Bus Stop,Bus Line
7,M1L,"Clairlea, Golden Mile, Oakridge",2,Coffee Shop,Grocery Store,Pizza Place,Burger Joint,Diner,Sandwich Place,Fast Food Restaurant,Pub,Food Truck,Bus Stop
10,M1P,"Dorset Park, Scarborough Town Centre, Wexford ...",2,Fast Food Restaurant,Coffee Shop,Grocery Store,Chinese Restaurant,Pizza Place,Wings Joint,Vietnamese Restaurant,Indian Restaurant,Pet Store,Burger Joint
11,M1R,"Maryvale, Wexford",2,Middle Eastern Restaurant,Coffee Shop,Grocery Store,Sandwich Place,Breakfast Spot,Restaurant,Filipino Restaurant,Asian Restaurant,Mediterranean Restaurant,Pizza Place
15,M1W,"L'Amoreaux West, Steeles West",2,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Sandwich Place,Tennis Court,Bakery,Pizza Place,Pool,Photography Studio,Japanese Restaurant


In [36]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[0]+[2] + list(range(5, scarborough_merged.shape[1]))]]

,Postalcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,"Rouge, Malvern",3,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Coffee Shop,Spa,Women's Store,Greek Restaurant,Fruit & Vegetable Store,Liquor Store,Movie Theater


In [37]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[0]+[2] + list(range(5, scarborough_merged.shape[1]))]]

,Postalcode,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,"Highland Creek, Rouge Hill, Port Union",4,Pizza Place,Italian Restaurant,Burger Joint,Breakfast Spot,Gym,Grocery Store,Hotel,Event Space,Falafel Restaurant,Electronics Store
